<a href="https://colab.research.google.com/github/R4HUL-ROY/Multimodal_feature_extraction/blob/main/Multimodal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.8
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

In [1]:
# !pip install -q tensorflow_text
# !pip install fasttext
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import pathlib
import tensorflow as tf
# import fasttext.util

# import tensorflow_hub as hub
# import tensorflow_text as text
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam, Adamax

In [2]:
merged_dataset_path = "/content/drive/MyDrive/project_resources/dataset/merged_data_for_multimodal_model.csv"
stopwords_path = "/content/drive/MyDrive/project_resources/stopwords/stopwords.txt"
glove_vector_path = '/content/drive/MyDrive/project_resources/glove_model/glove.6B.200d.txt'
fasttext_model_path = 'cc.en.300.bin'

vgg16_imagenet_weight_path = "/content/drive/MyDrive/project_resources/Imagenet_weights/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
resnet50_imagenet_weight_path = "/content/drive/MyDrive/project_resources/Imagenet_weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
mobilenetv2_imagenet_weight_path = "/content/drive/MyDrive/project_resources/Imagenet_weights/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5"

# Read the data

In [ ]:
# img_data_root = pathlib.Path('/content/drive/MyDrive/project_resources/dataset/Tobacco3482-jpg')
# print(img_data_root)
# for item in img_data_root.iterdir():
#   print(item)

# print()

# text_data_root = pathlib.Path('/content/drive/MyDrive/project_resources/dataset/tobaco_OCR')
# print(text_data_root)
# for item in text_data_root.iterdir():
#   print(item)

In [ ]:
# def get_corresponding_txtpath(img_path):
#     return img_path.replace("Tobacco3482-jpg", "tobaco_OCR")[:-3] + "txt"
    

# def get_file_paths_and_labels(img_data_root, text_data_root):
#      img_paths = [str(path) for path in img_data_root.glob('*/*.jpg')]
#      text_paths = [get_corresponding_txtpath(this_path) for this_path in img_paths]
#      img_labels = [p.split("/")[-2] for p in img_paths]
#      text_labels = [p.split("/")[-2] for p in text_paths]
#      return img_paths, img_labels, text_paths, text_labels

# img_paths, img_labels, text_paths, text_labels = get_file_paths_and_labels(img_data_root, text_data_root)
# print(len(img_paths))
# print(len(img_labels))
# print(len(text_paths))
# print(len(text_labels))

In [ ]:
# def get_text_from_path(path):
#     with open(path) as f:
#         lines = f.readlines()
#         lines  = ' '.join(lines)
#         f.close()
#     return lines

# out_text = get_text_from_path('/content/drive/MyDrive/tobaco_OCR/ADVE/0000435350.txt') 
# print(out_text)

In [ ]:
# text_contains = []
# c= 0 
# for i, this_path in enumerate(text_paths):
#     text_contains.append(get_text_from_path(this_path))
#     print(c, end= " ")
#     c +=1

In [ ]:
# df = pd.DataFrame(list(zip(text_paths, text_contains, img_paths, img_labels)),
#                columns =['text_paths','texts', 'img_paths', 'data_label'])

# # Merged Dataframe
# df.head()

In [ ]:
# df.to_csv("merged_data_for_multimodal_model.csv")

# Read dataset from csv

In [3]:
df = pd.read_csv(merged_dataset_path)
print(len(df))
df.head()

3482


,text_paths,texts,img_paths,data_label
0,/content/drive/MyDrive/project_resources/datas...,\n \n \n \n A Mpertant as yar\n sesiye teaeter...,/content/drive/MyDrive/project_resources/datas...,ADVE
1,/content/drive/MyDrive/project_resources/datas...,\n TE che fitm\n m66400 7127\n KOOLS are the o...,/content/drive/MyDrive/project_resources/datas...,ADVE
2,/content/drive/MyDrive/project_resources/datas...,SR Onrel ules cee\n Nee dss\n The one tales WT...,/content/drive/MyDrive/project_resources/datas...,ADVE
3,/content/drive/MyDrive/project_resources/datas...,so ARN Rr nr\n BWR Ga ||\n Vending Operators\n...,/content/drive/MyDrive/project_resources/datas...,ADVE
4,/content/drive/MyDrive/project_resources/datas...,\n &\n BR. :\n er non\n be 4\n op Re eo eee ee...,/content/drive/MyDrive/project_resources/datas...,ADVE


In [4]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['data_label']= le.fit_transform(df['data_label'])
df.head()

,text_paths,texts,img_paths,data_label
0,/content/drive/MyDrive/project_resources/datas...,\n \n \n \n A Mpertant as yar\n sesiye teaeter...,/content/drive/MyDrive/project_resources/datas...,0
1,/content/drive/MyDrive/project_resources/datas...,\n TE che fitm\n m66400 7127\n KOOLS are the o...,/content/drive/MyDrive/project_resources/datas...,0
2,/content/drive/MyDrive/project_resources/datas...,SR Onrel ules cee\n Nee dss\n The one tales WT...,/content/drive/MyDrive/project_resources/datas...,0
3,/content/drive/MyDrive/project_resources/datas...,so ARN Rr nr\n BWR Ga ||\n Vending Operators\n...,/content/drive/MyDrive/project_resources/datas...,0
4,/content/drive/MyDrive/project_resources/datas...,\n &\n BR. :\n er non\n be 4\n op Re eo eee ee...,/content/drive/MyDrive/project_resources/datas...,0


In [5]:
label_dict = {}
for idx, row in enumerate(df['text_paths']):
    path = df.at[idx, 'text_paths']
    lab = path.split("/")[-2]
    label_dict[str(lab)] = df.at[idx, 'data_label']
    
print(label_dict) 

{'ADVE': 0, 'Form': 2, 'Note': 6, 'Email': 1, 'News': 5, 'Resume': 8, 'Scientific': 9, 'Memo': 4, 'Report': 7, 'Letter': 3}


In [6]:
import re
def preprocess(text_string):
    preprocessed_string = re.sub(r'[^\w\s]','',text_string)
    preprocessed_string = preprocessed_string.replace('\n',' ')
    preprocessed_string = preprocessed_string.replace('_',' ')
    preprocessed_string = re.sub(' +', ' ', preprocessed_string)
    return preprocessed_string

In [7]:
# Tokenize, Lemmatize, stopwords removal
import spacy 
import nltk
# nlp = spacy.load("en", disable=['parser', 'tagger', 'ner'])
nlp = spacy.load("en_core_web_sm")
# from nltk.corpus import stopwords
# nltk.download('stopwords')
# stops = stopwords.words("english")

def get_stopwords(file_path):
    with open(file_path, "r") as fp:
        content = fp.read()
        stops = content.split("\n")
        stops = stops[:-1]
        fp.close()
        return stops

stops = get_stopwords(stopwords_path)
def normalize(comment, lowercase, remove_stopwords):
    if lowercase:
        comment = comment.lower()
    comment = nlp(comment)
    lemmatized = list()
    for word in comment:
        lemma = word.lemma_.strip()
        if lemma:
            if not remove_stopwords or (remove_stopwords and lemma not in stops):
                lemmatized.append(lemma)
    return " ".join(lemmatized)

normalize("counting playing the Home", lowercase=True, remove_stopwords=True)

'count play home'

In [8]:
df['texts'] = [preprocess(str(this_text)) for this_text in df['texts']]
df['texts'] = [normalize(this_text, lowercase=True, remove_stopwords=True) for this_text in df['texts']]
df.head()

,text_paths,texts,img_paths,data_label
0,/content/drive/MyDrive/project_resources/datas...,mpertant yar sesiye teaetere cabiieess baely k...,/content/drive/MyDrive/project_resources/datas...,0
1,/content/drive/MyDrive/project_resources/datas...,te che fitm m66400 7127 kool cigarette taste g...,/content/drive/MyDrive/project_resources/datas...,0
2,/content/drive/MyDrive/project_resources/datas...,sr onrel ules cee nee dss one tale wt lower ta...,/content/drive/MyDrive/project_resources/datas...,0
3,/content/drive/MyDrive/project_resources/datas...,arn rr nr bwr ga vend operator column worth 8 ...,/content/drive/MyDrive/project_resources/datas...,0
4,/content/drive/MyDrive/project_resources/datas...,br er non 4 op eo eee ee eee talk smoking deci...,/content/drive/MyDrive/project_resources/datas...,0


In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [10]:
def dataframe_to_dataset(dataframe):
    d = {}
    embed_tensor = []
    for i in dataframe['texts_embedding']:
        embed_tensor.append(tf.convert_to_tensor(i))

    img_path_tensor= []
    for i in dataframe['img_paths']:
        img_path_tensor.append(tf.convert_to_tensor(i))

    d['texts_embedding'] = embed_tensor
    d['img_paths'] = img_path_tensor

    labels = dataframe["data_label"]
    labels = tf.convert_to_tensor(labels)
    ds = tf.data.Dataset.from_tensor_slices((d, labels))
    return ds

In [11]:
@tf.function
def preprocess_image(image_path):
    extension = tf.strings.split(image_path)[-1]

    image = tf.io.read_file(image_path)
    if extension == b"jpg":
        image = tf.image.decode_jpeg(image, 3)
    else:
        image = tf.image.decode_png(image, 3)
    image = tf.image.resize(image, (224, 224))
    image = tf.image.per_image_standardization(image)
    return image

@tf.function
def preprocess_text(sample):
    pass

@tf.function
def preprocess_text_and_image(sample):
    image = preprocess_image(sample["img_paths"])
    text = sample['texts_embedding']
    return {"image_inputs": image,  "text_inputs": text}

In [12]:
batch_size = 64
auto = tf.data.AUTOTUNE

def prepare_dataset(dataframe, training = True):
    ds = dataframe_to_dataset(dataframe)
    if training:
        ds = ds.shuffle(len(train_df))
    ds = ds.map(lambda x, y: (preprocess_text_and_image(x), y)).cache()
    ds = ds.batch(batch_size).prefetch(auto)
    return ds

In [13]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Flatten,  Dense, Dropout, Conv1D, GlobalMaxPooling1D, MaxPooling1D, GlobalMaxPooling2D
from keras.initializers import Constant
from tensorflow.keras.optimizers import Adam

In [14]:
def get_embedding_matrix(text_model):
    if text_model == "glove":
        emmbed_dict = {}
        with open(glove_vector_path ,'r') as f:
            for line in f:
                values = line.split()
                word = values[0]
                vector = np.asarray(values[1:],'float32')
                emmbed_dict[word]=vector
            f.close()
        embedding_matrix = np.zeros((num_words, 200))
        for word, i in word_index.items():
            if i < num_words:
                emb_vec = emmbed_dict.get(word)
                if emb_vec is not None:
                    embedding_matrix[i] = emb_vec
        return embedding_matrix 

    elif text_model == "fasttext":  
        ft = fasttext.load_model(fasttext_model_path)    
        embedding_matrix = np.zeros((num_words, 300)) 
        for word, i in word_index.items():
            if i < num_words:
                emb_vec = ft.get_word_vector(word)
                if emb_vec is not None:
                    embedding_matrix[i] = emb_vec 
        return embedding_matrix    
    else:    
        return None                                       

In [15]:
def create_text_encoder(text_projection_dims, dropout_rate, text_model, text_model_trainable):

    embedding_matrix = get_embedding_matrix(text_model)

    filter_size = 128
    stride_unit = 1
    kernels = 3
    dropout_rate = 0.2
    dropout_rate_conv = 0.5

    inputs = keras.Input(shape=(500,), dtype=tf.int32, name="text_inputs")

    if text_model == "glove":
        embed = tf.keras.layers.Embedding(  num_words,
                                            200,
                                            input_length = 500,
                                            embeddings_initializer = Constant(embedding_matrix),
                                            trainable = text_model_trainable
                                        )(inputs)
    if text_model == "fasttext":
        embed = tf.keras.layers.Embedding(  num_words,
                                            300,
                                            input_length = 500,
                                            embeddings_initializer = Constant(embedding_matrix),
                                            trainable = text_model_trainable
                                        )(inputs)

    x = tf.keras.layers.Dropout(dropout_rate)(embed)


    x = tf.keras.layers.Conv1D(filters=filter_size,kernel_size=kernels, padding='valid',activation='relu',strides=stride_unit,
                               kernel_initializer = "glorot_uniform" ,bias_initializer = 'zeros')(x)
    x = tf.keras.layers.MaxPooling1D()(x)
    x = tf.keras.layers.Dropout(dropout_rate_conv)(x)

    x = tf.keras.layers.Conv1D(filters=filter_size,kernel_size=kernels, padding='valid',activation='relu',strides=stride_unit,
                               kernel_initializer = "glorot_uniform" ,bias_initializer = 'zeros')(x)
    x = tf.keras.layers.MaxPooling1D()(x)
    x = tf.keras.layers.Dropout(dropout_rate_conv)(x)

    x = tf.keras.layers.Conv1D(filters=filter_size,kernel_size=kernels, padding='valid',activation='relu',strides=stride_unit,
                               kernel_initializer = "glorot_uniform" ,bias_initializer = 'zeros')(x)
    x = tf.keras.layers.MaxPooling1D()(x)
    x = tf.keras.layers.Dropout(dropout_rate_conv)(x)
    
    x = tf.keras.layers.Conv1D(filters=filter_size,kernel_size=kernels, padding='valid',activation='relu',strides=stride_unit,
                               kernel_initializer = "glorot_uniform" ,bias_initializer = 'zeros')(x)
    x = tf.keras.layers.MaxPooling1D()(x)
    x = tf.keras.layers.Dropout(dropout_rate_conv)(x)

    x = tf.keras.layers.Flatten()(x)
    # x = tf.math.l2_normalize(x)
    x = tf.keras.layers.Dropout(dropout_rate_conv)(x)
     
    outputs = keras.layers.Dense(units=text_projection_dims,activation="relu",kernel_initializer="glorot_uniform",bias_initializer='zeros')(x)
    return keras.Model(inputs, outputs, name="text_encoder")

In [16]:
def create_vision_encoder(img_projection_dims, dropout_rate, image_model, img_base_model_trainable):
    img_shape=(224,224,3)
    seed_value = 42


    if image_model == "vgg16":
        base_model = tf.keras.applications.vgg16.VGG16(include_top=False,input_shape=img_shape,pooling='avg',classes=10,weights=None)
        base_model.load_weights(vgg16_imagenet_weight_path)
        base_model.trainable = img_base_model_trainable

    elif image_model == "resnet50":
        base_model= tf.keras.applications.ResNet50(include_top=False,input_shape=img_shape,pooling='avg',classes=10,weights=None)
        base_model.load_weights(resnet50_imagenet_weight_path)
        base_model.trainable = img_base_model_trainable

    elif image_model == "mobilenetv2":
        base_model=tf.keras.applications.MobileNetV2(include_top=False,input_shape=img_shape,pooling='avg', classes=10,weights=None)
        base_model.load_weights(mobilenetv2_imagenet_weight_path)
        base_model.trainable = img_base_model_trainable


    inputs = keras.Input(shape=img_shape)
    x = base_model(inputs, training = img_base_model_trainable)
    outputs =tf.keras.layers.Dense(img_projection_dims,kernel_initializer ="glorot_uniform",bias_initializer= "zeros",activation='relu')(x)

    model=Model(inputs, outputs)

    # Receive the images as inputs.
    image_inputs = keras.Input(shape=(224, 224, 3), name="image_inputs")

    if image_model == "vgg16":
        preprocessed_image = tf.keras.applications.vgg16.preprocess_input(image_inputs)
    if image_model == "resnet50":
        preprocessed_image = tf.keras.applications.resnet50.preprocess_input(image_inputs)    
    if image_model == "mobilenetv2":
        preprocessed_image = tf.keras.applications.mobilenet_v2.preprocess_input(image_inputs)

    embeddings = model(preprocessed_image)

    # Create the vision encoder model.
    return keras.Model(image_inputs, embeddings, name="vision_encoder")

In [17]:
def create_multimodal_model(**kwargs):

    img_projection_dims = kwargs['img_projection_dims']
    text_projection_dims = kwargs['text_projection_dims']
    dropout_rate = kwargs['dropout_rate']
    image_model = kwargs['image_model']
    text_model = kwargs['text_model']
    img_base_model_trainable = kwargs['img_base_model_trainable']
    text_model_trainable = kwargs['text_model_trainable']

    if image_model == "None" and text_model == "None":
        print("Both the image_model and text_model cannot be None at the same time !!")
        return
    # Receive the images and text as inputs.
    if image_model != "None":
        image_inputs = keras.Input(shape=(224, 224, 3), name="image_inputs")
        vision_encoder = create_vision_encoder(img_projection_dims, dropout_rate, image_model, img_base_model_trainable)
        vision_projections = vision_encoder(image_inputs)

    if text_model != "None":   
        text_inputs = keras.Input(shape=(500,), dtype=tf.int32, name="text_inputs")
        text_encoder = create_text_encoder(text_projection_dims, dropout_rate, text_model, text_model_trainable)
        text_projections = text_encoder(text_inputs)


    # Concatenate the projections and pass through the classification layer.
    if image_model != "None" and text_model != "None":
        concatenated = keras.layers.Concatenate()([vision_projections, text_projections])
    # concatenated = tf.keras.layers.BatchNormalization(momentum=0.9)(concatenated)


    if image_model == "None" and text_model != "None":
        outputs = keras.layers.Dense(10, activation="softmax")(text_projections)
        return keras.Model(text_inputs, outputs)
    elif text_model == "None" and image_model != "None":    
        outputs = keras.layers.Dense(10, activation="softmax")(vision_projections)
        return keras.Model(image_inputs, outputs)
    elif text_model != "None" and image_model != "None":
        outputs = keras.layers.Dense(10, activation="softmax")(concatenated)
        return keras.Model([image_inputs, text_inputs], outputs)
    else:
        return None

    

In [18]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import random
random.seed(56)

In [19]:
epochs = 50
fine_tune_epochs = 5
lr = 0.001

d = {
    "precision" : [],
     "recall" : [],
     "f1_score" : [],
     "accuracy" : [],
     "seed_value" : []
}
seed = []

for i in range(1):
    print('#Run: ', i+1)
    sd = random.randint(0, 100)
    seed.append(sd)

    # Split the dataframe
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=sd)
    test_df, val_df = train_test_split(test_df, test_size=0.5, random_state=sd)

    max_len = 500
    # Fit the tokenizer
    tokenizer = Tokenizer(num_words=65000)
    tokenizer.fit_on_texts(train_df['texts'])

    # sequence the input corpus and add zero padding upto 500 word
    train_sequence = tokenizer.texts_to_sequences(train_df['texts'])
    train_padded = pad_sequences(train_sequence, maxlen = max_len, truncating = "post", padding = "post" )

    valid_sequence = tokenizer.texts_to_sequences(val_df['texts'])
    valid_padded = pad_sequences(valid_sequence, maxlen = max_len, truncating = "post", padding = "post" )

    test_sequence = tokenizer.texts_to_sequences(test_df['texts'])
    test_padded = pad_sequences(test_sequence, maxlen = max_len, truncating = "post", padding = "post" )

    train_tensor = [tf.convert_to_tensor(train_padded[i]) for i in range(train_padded.shape[0])]
    train_df['texts_embedding'] = train_tensor

    test_tensor = [tf.convert_to_tensor(test_padded[i]) for i in range(test_padded.shape[0])]
    test_df['texts_embedding'] = test_tensor   

    val_tensor = [tf.convert_to_tensor(valid_padded[i]) for i in range(valid_padded.shape[0])]
    val_df['texts_embedding'] = val_tensor

    # Preparing dataset
    train_ds = prepare_dataset(train_df)
    val_ds = prepare_dataset(val_df, False)
    test_ds = prepare_dataset(test_df, False)

    word_index = tokenizer.word_index
    num_words = len(word_index) + 1

    """
    options for image_model : ["vgg16", "resnet50", "mobilenetv2"] (case-sensitive input)
    options for text_model : ["glove", "fasttext"] (case-sensitive input)
    """

    multimodal_model = create_multimodal_model( 
        img_projection_dims=512,
        text_projection_dims=512,
        dropout_rate=0.2,
        image_model = "None",
        text_model = "glove", 
        img_base_model_trainable=False,
        text_model_trainable= False
    )

    multimodal_model.compile(Adamax(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics="accuracy")
    history = multimodal_model.fit(train_ds, validation_data=val_ds, epochs=10, verbose=2)


#Run:  1
Epoch 1/10


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:559: UserWarning: Input dict contained keys ['text_inputs'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


44/44 - 1707s - loss: 1.7982 - accuracy: 0.4036 - val_loss: 1.5649 - val_accuracy: 0.4413 - 1707s/epoch - 39s/step
Epoch 2/10


KeyboardInterrupt: ignored

In [ ]:
keras.utils.plot_model(multimodal_model, show_shapes=True)

In [ ]:
history = multimodal_model.fit(train_ds, validation_data=val_ds, epochs=50, verbose=2)

Epoch 1/50


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:559: UserWarning: Input dict contained keys ['image_inputs'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


KeyboardInterrupt: ignored

In [ ]:
model.save('/content')

In [ ]:
# multimodal_model.compile(Adamax(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics="accuracy")

In [ ]:
# multimodal_model.layers

In [ ]:
# history_fine = multimodal_model.fit(train_ds, validation_data=val_ds, epochs=60, verbose=2)
# 
# y_pred = multimodal_model.predict(test_ds)
# y_pred = [np.argmax(i) for i in y_pred]

# y_true = []
# for element in test_ds:
#     y_true.extend(list(element[1].numpy()))

Epoch 1/70
44/44 - 1s - loss: 0.6351 - accuracy: 0.7767 - val_loss: 0.7052 - val_accuracy: 0.7564 - 989ms/epoch - 22ms/step
Epoch 2/70
44/44 - 1s - loss: 0.6031 - accuracy: 0.7741 - val_loss: 0.7010 - val_accuracy: 0.7450 - 996ms/epoch - 23ms/step
Epoch 3/70
44/44 - 1s - loss: 0.6015 - accuracy: 0.7835 - val_loss: 0.7074 - val_accuracy: 0.7393 - 935ms/epoch - 21ms/step
Epoch 4/70
44/44 - 1s - loss: 0.6125 - accuracy: 0.7871 - val_loss: 0.7120 - val_accuracy: 0.7421 - 1s/epoch - 24ms/step
Epoch 5/70
44/44 - 1s - loss: 0.5775 - accuracy: 0.7903 - val_loss: 0.6916 - val_accuracy: 0.7421 - 982ms/epoch - 22ms/step
Epoch 6/70
44/44 - 1s - loss: 0.5801 - accuracy: 0.7896 - val_loss: 0.6914 - val_accuracy: 0.7479 - 948ms/epoch - 22ms/step
Epoch 7/70
44/44 - 1s - loss: 0.5661 - accuracy: 0.7925 - val_loss: 0.7010 - val_accuracy: 0.7479 - 984ms/epoch - 22ms/step
Epoch 8/70
44/44 - 1s - loss: 0.5634 - accuracy: 0.8043 - val_loss: 0.6943 - val_accuracy: 0.7536 - 972ms/epoch - 22ms/step
Epoch 9/70
